# Short Notebook
#### TDT4173 MACHINE LEARNING // Fall 2021

#### Participants: 
- Helle Villmones Haug, student ID: 507372
- Erik Galler, student ID: 529208   
- Beatrice Kiær, student ID: 477864

#### Kaggle competition name: Fit Check (Group 55)
#### Kaggle team name: Fit Check (Group 55)



### ----------------------------------------------------------------------------------------------------------------------------------

This notebook replicates the Kaggle submission with score 0.15010 (submission (99).csv) on the public leaderboard. The same notebook can be use to replicate the Kaggle submission with score 0.15095 (submission (68).csv); The only difference is the hyperparameters in RandomForestRegressor, LGBMRegressor and CatBoostRegressor, in addition to learning rate in the meta model. The changes are commented in box 38 and box 42, and are the only differences between the two submissions.

***NB!*** The Pandas/Numpy installations below were necessary to run Datawig and the other Pandas/Numpy dependent libraries. 

In [1]:
#pip install pandas==0.25.3

Note: you may need to restart the kernel to use updated packages.


In [2]:
#pip install numpy==1.19.5

Note: you may need to restart the kernel to use updated packages.


In [3]:
import math
import numpy as np 
import pandas as pd
import mxnet as mx
import scipy
import datawig
import time
import seaborn as sns
import geopy.distance
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import LocalOutlierFactor
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import boxcox 
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from data import Data;

## Load data: 

In [4]:
start_time = time.time();

data = Data()
data.open()
df_train = data.train
df_test = data.test
df_all = data.all

df_train = df_train.sort_index()
df_test = df_test.sort_index()

Loaded 23285 training samples
Loaded 9937 test samples 



## Price transformation

In [5]:
df_train['price'] = df_train['price']/df_train['area_total'];
df_train['price'] = df_train['price'].apply(lambda x: np.log10(x));

2021-11-19 12:57:31,112 [INFO]  NumExpr defaulting to 8 threads.


## Omitting columns

In [6]:
train = df_train.drop(['address'], axis=1);
test = df_test.drop(['address'], axis=1);

## Fixing nonsensicle rows

### Latitude and Longitude

In [7]:
display(test.loc[((test['latitude'] < 55.2) | (df_test['latitude']> 56.1))][['longitude', 'latitude']])
test.loc[((test['latitude'] < 55.2) | (test['latitude'] > 56.1)) & (df_test['street'] =='Бунинские Луга ЖК' ),
         ['longitude', 'latitude']] = [37.478055, 55.544046];

mean = test.loc[((test['latitude'] > 55.2) & (test['latitude'] < 56.1)) & (df_test['street'] == 'улица Центральная'),
         ['longitude', 'latitude']].mean().values;

test.loc[((test['latitude'] < 55.2) | (test['latitude']> 56.1)) & (df_test['street'] == 'улица Центральная'),
         ['longitude', 'latitude']] = mean;

test.loc[df_test['street'] == 'улица 1-я Линия', ['latitude', 'longitude']] = [55.642338, 37.3789709]

,longitude,latitude
2511,-61.790500,17.141734
2529,74.517184,42.914700
4719,132.768097,43.353545
5090,-61.790500,17.141734
6959,-61.790500,17.141734
8596,-61.790500,17.141734
9547,132.759204,43.352968


### Ceiling

In [8]:
train.loc[(train['ceiling'] == 0), ['ceiling']] = np.nan
test.loc[(train['ceiling'] == 0), ['ceiling']] = np.nan
train['ceiling'] = train['ceiling'].fillna(200)
test['ceiling'] = test['ceiling'].fillna(200)

train['ceiling'] = train['ceiling'].apply(lambda h: h*100 if h < 100 else h)
test['ceiling'] = test['ceiling'].apply(lambda h: h*100 if h < 100 else h)

## Missing values

In [9]:
meta_lst = data.describe_data(data.apartments_train, data.apartments_meta);
meta_lst += data.describe_data(data.buildings_train, data.buildings_meta);

meta={}
for attribute in meta_lst:
    meta[attribute['name']] = {'type': attribute['type'], 'nan': df_train[attribute['name']].isna().sum()};

,Type,#NaN,Description,Statistics
id,integer_id,0,Unique ID for apartment,#unique=23285
seller,categorical,8830,The type of apartment seller,"Agents(1895), Company(4768), Developer(6185), Owner(1607)"
price,real,0,The listed price of the apartment (TARGET),"mean=23556173.5, std=52643927.8, min=900000.0, max=2600000000.0"
area_total,real,0,Total area of the apartment,"mean=74.5, std=58.7, min=9.3, max=2181.0"
area_kitchen,real,4721,Total kitchen area in the apartment,"mean=12.6, std=6.6, min=1.0, max=100.0"
area_living,real,3882,Total living space area in the apartment,"mean=38.7, std=31.4, min=0.0, max=900.0"
floor,integer,0,Primary building floor of the apartment,"mean=9.0, std=8.3, min=1.0, max=95.0"
rooms,integer,0,Number of rooms in the apartment,"mean=2.2, std=1.1, min=1.0, max=6.0"
layout,categorical,17642,Overal apartment layout,"Adjacent(241), Adjacent_isolated(465), Isolated(4937)"
ceiling,real,11093,Ceiling height in the apartment,"mean=3.3, std=10.8, min=0.0, max=340.0"


,Type,#NaN,Description,Statistics
id,integer_id,0,Unique ID of building for joining with apartments,#unique=6791
new,boolean,227,Whether it is an old or new building,"False(6141), True(423)"
latitude,real,0,Latitude coordinate of building,"mean=55.7, std=0.1, min=55.2, max=56.0"
longitude,real,0,Longitude coordinate of building,"mean=37.6, std=0.2, min=36.9, max=38.0"
district,categorical,1,Administriative district within Moscow,"Central(637), East(956), North(593), North-East(630), North-West(553), Novomoskovsk(344), South(758), South-East(672), South-West(900), Troitsk(146), West(535), Zelenograd(66)"
street,string,0,Bulding street name,#unique=1682
address,string,0,Building address (within street),#unique=1851
constructed,integer,479,Year when the building was constructed,"mean=1985.8, std=24.3, min=1855.0, max=2023.0"
material,categorical,1165,Primary building material used in building,"Block(679), Bricks(1226), Monolith(1312), Monolithic_brick(19), Panel(2384), Stalin_project(2), Wood(4)"
stories,integer,0,Total number of floors in the building,"mean=13.0, std=7.0, min=1.0, max=95.0"


### Latitude/Longitude

In [10]:
# Missing value for longitude and latitude, street: пос. Коммунарка on both
mean_lat = df_test.loc[df_test['street'] == 'пос. Коммунарка']['latitude'].mean()
mean_lon = df_test.loc[df_test['street'] == 'пос. Коммунарка']['longitude'].mean()
test['latitude'] = test['latitude'].fillna(mean_lat)
test['longitude'] = test['longitude'].fillna(mean_lon)

### Street


In [11]:
train_street = train['street'].unique()
test_street = test['street'].unique()
mask = np.isin(test_street, train_street)
overlapping_streets = test_street[mask]

for index, row in test.iterrows():
    if row['street'] in train_street:
        test.loc[index, 'street'] = np.where(train_street == row['street'])[0][0]
    else:
        train_street = np.append(train_street, row['street'])
        test.loc[index, 'street'] = np.where(train_street == row['street'])[0][0]
    
for index, row in train.iterrows():
    train.loc[index, 'street'] = np.where(train_street == row['street'])[0][0]

In [12]:
train['seller'] = train['seller'].fillna(4)
test['seller'] = test['seller'].fillna(4)

train['parking'] = train['parking'].fillna(3)
test['parking'] = test['parking'].fillna(3)

train['elevator_without'] = train['elevator_without'].fillna(0)
test['elevator_without'] = test['elevator_without'].fillna(0)

train['elevator_passenger'] = train['elevator_passenger'].fillna(0) 
test['elevator_passenger'] = test['elevator_passenger'].fillna(0) 

train['elevator_service'] = train['elevator_service'].fillna(0)
test['elevator_service'] = test['elevator_service'].fillna(0)

train['balconies'] = train['balconies'].fillna(0)
test['balconies'] = test['balconies'].fillna(0)

train['loggias'] = train['loggias'].fillna(0)
test['loggias'] = test['loggias'].fillna(0)

train['bathrooms_shared'] = train['bathrooms_shared'].fillna(0)
test['bathrooms_shared'] = test['bathrooms_shared'].fillna(0)

train['bathrooms_private'] = train['bathrooms_private'].fillna(1)
test['bathrooms_private'] = test['bathrooms_private'].fillna(1)

train['condition'] = train['condition'].fillna(4)
test['condition'] = test['condition'].fillna(4)

train['layout'] = train['layout'].fillna(3)
test['layout'] = test['layout'].fillna(3)

train['garbage_chute'] = train['garbage_chute'].fillna(train['elevator_passenger'])
test['garbage_chute'] = test['garbage_chute'].fillna(test['elevator_passenger'])

### District

In [13]:
def find_closest_row(df, attribute):
    start_id = df.iloc[0]['id']
    coordinates = df[['latitude', 'longitude']]*100000
    missing_test_district = df.loc[np.isnan(df['district'])][['latitude', 'longitude']]*100000
    ary = scipy.spatial.distance.cdist(coordinates, missing_test_district, metric='euclidean')
    min_dist = pd.DataFrame(ary)
    for i, row in enumerate(missing_test_district.itertuples()):
        missing_row_index = missing_test_district.iloc[[i]].index.values[0]
        min_dist_series = min_dist[i]
        closest = min_dist_series.nsmallest(300).index.values
        for neighboor in closest:
            district_row = df_train.loc[df_train['id'] == neighboor]
            if not np.isnan(district_row['district'].values[0]):
                district = district_row['district'].values[0]
                df.loc[df['id'] == missing_row_index + start_id, ['district']] = district
                break;
                
find_closest_row(train, 'district');
find_closest_row(test, 'district');

### Material

In [14]:
def find_closest_row(df, n):
    start_id = df.iloc[0]['id']
    coordinates = df[['latitude', 'longitude']]*100000
    missing_test_district = df.loc[np.isnan(df['material'])][['latitude', 'longitude']]*100000
    ary = scipy.spatial.distance.cdist(coordinates, missing_test_district, metric='euclidean')
    min_dist = pd.DataFrame(ary)
    for i, row in enumerate(missing_test_district.itertuples()):
        missing_row_index = missing_test_district.iloc[[i]].index.values[0]
        min_dist_series = min_dist[i]
        closest = min_dist_series.nsmallest(1000).index.values
        materials = [np.nan]*n
        j = 0
        for i, neighboor in enumerate(closest):
            district_row = df_train.loc[df_train['id'] == neighboor]
            if not np.isnan(district_row['material'].values[0]):
                material = int(district_row['material'].values[0])
                materials[j] = material
                j += 1
                if j == n:
                    break;
        most_frequent = np.bincount(materials).argmax()
        df.loc[df['id'] == missing_row_index + start_id, ['material']] = most_frequent
        
find_closest_row(train, 15);
find_closest_row(test, 15);

### Constructed

In [15]:
storcon = train.groupby('stories')['constructed'].agg('mean').to_frame()
storcon = storcon.reset_index().round()
#Fill "constructed" values with mean of next and previous value
storcon = (storcon.ffill()+storcon.bfill())/2
#If first value isna, fill with next
storcon = storcon.fillna(method='ffill')
#If last value isna, fill with previous
storcon = storcon.fillna(method='bfill')

for index, row in train.iterrows():
    if pd.isna(row['constructed']):
        dist = min(abs(storcon['stories'] - row['stories']))
       
        if (row['stories'] + dist) in storcon.stories:
            val = storcon.loc[storcon['stories'] == row['stories'] + dist, 'constructed'].iloc[0]
            train.loc[index, 'constructed'] = val
        else:
            val = storcon.loc[storcon['stories'] == row['stories'] + dist, 'constructed'].iloc[0]
            train.loc[index, 'constructed'] = val
            
for index, row in test.iterrows():
    if pd.isna(row['constructed']):
        dist = min(abs(storcon['stories'] - row['stories']))
       
        if (row['stories'] + dist) in storcon.stories:
            val = storcon.loc[storcon['stories'] == row['stories'] + dist, 'constructed'].iloc[0]
            test.loc[index, 'constructed'] = val
        else:
            val = storcon.loc[storcon['stories'] == row['stories'] + dist, 'constructed'].iloc[0]
            test.loc[index, 'constructed'] = val 

## Remaining Missing Values with Datawig

In [16]:
omit_cols = ['id', 'building_id', 'street', 'address', 'price']
nan_cols = test.columns[test.isna().any()].tolist();
N = 7
ctx = mx.cpu();

for i, nan_col in enumerate(nan_cols):
    print(nan_col)
    print(f"Attribute ({i+1}/{len(nan_cols)})")
    
    c_train = train.corr().abs().nlargest(N, nan_col)[nan_col].index;
    if 'price' in c_train:
        c_train = c_train.drop(pd.Index(['price']))
    c_train = c_train.drop(pd.Index([nan_col])); 

    print("Most correlated:", c_train.tolist())
    if nan_col not in omit_cols:
        imputer = datawig.SimpleImputer(
            input_columns=c_train, # column(s) containing information about the column we want to impute
            output_column=nan_col, # the column we'd like to impute values for
            output_path = 'datawig_imputer' # stores model data and metrics
            )
        # Fit an imputer model on the train data
        imputer.fit(train_df=train, ctx=ctx,
                   learning_rate=4e-2,
                    #num_epochs=20,
                    patience=3,
                    test_split=0.1,
                    weight_decay=0.1)

        # Impute missing values and return original dataframe with predictions
        imputed_col = nan_col + "_imputed"
        
        imputed_test = imputer.predict(test)
        imputed_train = imputer.predict(train)
        
        test.loc[np.isnan(test[nan_col]), [nan_col]] = imputed_test.loc[np.isnan(test[nan_col])][imputed_col]
        train.loc[np.isnan(train[nan_col]), [nan_col]] = imputed_train.loc[np.isnan(train[nan_col])][imputed_col]

    # Categorical
    if meta[nan_col]['type'] != 'real':
        train[nan_col] = train[nan_col].round().astype(int)
        test[nan_col] = test[nan_col].round().astype(int)

area_kitchen
Attribute (1/7)
Most correlated: ['area_total', 'area_living', 'constructed', 'parking', 'rooms', 'elevator_service']


[13:01:01] ../src/executor/graph_executor.cc:1995: Subgraph backend MKLDNN is activated.
2021-11-19 13:01:01,341 [INFO]  
========== start: fit model
2021-11-19 13:01:01,343 [WARNING]  Already bound, ignoring bind()
2021-11-19 13:01:01,842 [INFO]  Epoch[0] Batch [0-655]	Speed: 22126.79 samples/sec	cross-entropy=11.808159	area_kitchen-accuracy=0.000000
2021-11-19 13:01:02,297 [INFO]  Epoch[0] Train-cross-entropy=12.369714
2021-11-19 13:01:02,298 [INFO]  Epoch[0] Train-area_kitchen-accuracy=0.000000
2021-11-19 13:01:02,299 [INFO]  Epoch[0] Time cost=0.952
2021-11-19 13:01:02,315 [INFO]  Saved checkpoint to "datawig_imputer/model-0000.params"
2021-11-19 13:01:02,371 [INFO]  Epoch[0] Validation-cross-entropy=13.993332
2021-11-19 13:01:02,372 [INFO]  Epoch[0] Validation-area_kitchen-accuracy=0.000000
2021-11-19 13:01:02,895 [INFO]  Epoch[1] Batch [0-655]	Speed: 20123.92 samples/sec	cross-entropy=11.504424	area_kitchen-accuracy=0.000000
2021-11-19 13:01:03,412 [INFO]  Epoch[1] Train-cross-en

area_living
Attribute (2/7)
Most correlated: ['area_total', 'rooms', 'area_kitchen', 'bathrooms_shared', 'windows_street']


[13:01:16] ../src/executor/graph_executor.cc:1995: Subgraph backend MKLDNN is activated.
2021-11-19 13:01:16,979 [INFO]  
========== start: fit model
2021-11-19 13:01:16,980 [WARNING]  Already bound, ignoring bind()
2021-11-19 13:01:17,431 [INFO]  Epoch[0] Batch [0-655]	Speed: 23585.82 samples/sec	cross-entropy=9.242701	area_living-accuracy=0.000000
2021-11-19 13:01:17,885 [INFO]  Epoch[0] Train-cross-entropy=9.465264
2021-11-19 13:01:17,886 [INFO]  Epoch[0] Train-area_living-accuracy=0.000000
2021-11-19 13:01:17,887 [INFO]  Epoch[0] Time cost=0.902
2021-11-19 13:01:17,895 [INFO]  Saved checkpoint to "datawig_imputer/model-0000.params"
2021-11-19 13:01:17,943 [INFO]  Epoch[0] Validation-cross-entropy=7.756526
2021-11-19 13:01:17,944 [INFO]  Epoch[0] Validation-area_living-accuracy=0.000000
2021-11-19 13:01:18,437 [INFO]  Epoch[1] Batch [0-655]	Speed: 21349.15 samples/sec	cross-entropy=8.753687	area_living-accuracy=0.000000
2021-11-19 13:01:18,908 [INFO]  Epoch[1] Train-cross-entropy=9.

windows_court
Attribute (3/7)
Most correlated: ['windows_street', 'new', 'stories', 'constructed', 'floor', 'elevator_service']


...

[13:01:43] ../src/executor/graph_executor.cc:1995: Subgraph backend MKLDNN is activated.
2021-11-19 13:01:43,039 [INFO]  
========== start: fit model
2021-11-19 13:01:43,040 [WARNING]  Already bound, ignoring bind()
2021-11-19 13:01:43,523 [INFO]  Epoch[0] Batch [0-655]	Speed: 21954.12 samples/sec	cross-entropy=14.311210	phones-accuracy=0.000000
2021-11-19 13:01:43,981 [INFO]  Epoch[0] Train-cross-entropy=13.933518
2021-11-19 13:01:43,982 [INFO]  Epoch[0] Train-phones-accuracy=0.000000
2021-11-19 13:01:43,983 [INFO]  Epoch[0] Time cost=0.939
2021-11-19 13:01:43,991 [INFO]  Saved checkpoint to "datawig_imputer/model-0000.params"
2021-11-19 13:01:44,036 [INFO]  Epoch[0] Validation-cross-entropy=14.089022
2021-11-19 13:01:44,038 [INFO]  Epoch[0] Validation-phones-accuracy=0.000000
2021-11-19 13:01:44,518 [INFO]  Epoch[1] Batch [0-655]	Speed: 21945.49 samples/sec	cross-entropy=14.086751	phones-accuracy=0.000000
2021-11-19 13:01:45,028 [INFO]  Epoch[1] Train-cross-entropy=13.894989
2021-11-

new
Attribute (6/7)
Most correlated: ['condition', 'constructed', 'layout', 'parking', 'elevator_service', 'area_kitchen']


[13:01:51] ../src/executor/graph_executor.cc:1995: Subgraph backend MKLDNN is activated.
2021-11-19 13:01:51,181 [INFO]  
========== start: fit model
2021-11-19 13:01:51,183 [WARNING]  Already bound, ignoring bind()
2021-11-19 13:01:51,636 [INFO]  Epoch[0] Batch [0-655]	Speed: 23369.12 samples/sec	cross-entropy=6.139277	new-accuracy=0.192073
2021-11-19 13:01:52,090 [INFO]  Epoch[0] Train-cross-entropy=5.784816
2021-11-19 13:01:52,091 [INFO]  Epoch[0] Train-new-accuracy=0.180428
2021-11-19 13:01:52,092 [INFO]  Epoch[0] Time cost=0.907
2021-11-19 13:01:52,099 [INFO]  Saved checkpoint to "datawig_imputer/model-0000.params"
2021-11-19 13:01:52,149 [INFO]  Epoch[0] Validation-cross-entropy=5.737146
2021-11-19 13:01:52,150 [INFO]  Epoch[0] Validation-new-accuracy=0.000000
2021-11-19 13:01:52,636 [INFO]  Epoch[1] Batch [0-655]	Speed: 21688.01 samples/sec	cross-entropy=5.570494	new-accuracy=0.192073
2021-11-19 13:01:53,164 [INFO]  Epoch[1] Train-cross-entropy=5.717339
2021-11-19 13:01:53,165 [

heating
Attribute (7/7)
Most correlated: ['constructed', 'district', 'latitude', 'longitude', 'area_kitchen', 'elevator_without']


[13:01:58] ../src/executor/graph_executor.cc:1995: Subgraph backend MKLDNN is activated.
2021-11-19 13:01:58,193 [INFO]  
========== start: fit model
2021-11-19 13:01:58,194 [WARNING]  Already bound, ignoring bind()
2021-11-19 13:01:58,714 [INFO]  Epoch[0] Batch [0-655]	Speed: 20370.15 samples/sec	cross-entropy=15.080270	heating-accuracy=0.000000
2021-11-19 13:01:59,170 [INFO]  Epoch[0] Train-cross-entropy=15.270702
2021-11-19 13:01:59,172 [INFO]  Epoch[0] Train-heating-accuracy=0.000000
2021-11-19 13:01:59,174 [INFO]  Epoch[0] Time cost=0.976
2021-11-19 13:01:59,181 [INFO]  Saved checkpoint to "datawig_imputer/model-0000.params"
2021-11-19 13:01:59,216 [INFO]  Epoch[0] Validation-cross-entropy=22.756283
2021-11-19 13:01:59,217 [INFO]  Epoch[0] Validation-heating-accuracy=0.000000
2021-11-19 13:01:59,711 [INFO]  Epoch[1] Batch [0-655]	Speed: 21343.41 samples/sec	cross-entropy=14.901001	heating-accuracy=0.000000
2021-11-19 13:02:00,216 [INFO]  Epoch[1] Train-cross-entropy=15.260895
2021

## Kitchen and Living engineering

In [17]:
train["living_to_total"] = train.apply(lambda row: row['area_living'] / row['area_total'], axis=1);
test["living_to_total"] = test.apply(lambda row: row['area_living'] / row['area_total'], axis=1);

## Distance engineering

In [18]:
def calc_distance(df, site_coords):
    station_coords = (df['latitude'], df['longitude'])
    d = geopy.distance.distance(site_coords, station_coords).km
    return(d)

In [19]:
center_lat = 55.7558
center_lon = 37.6173
train['distance'] = train.apply(calc_distance, site_coords=(center_lat, center_lon), axis=1)
test['distance'] = test.apply(calc_distance, site_coords=(center_lat, center_lon), axis=1)

## Theta engineering

In [20]:
 def calc_degrees(df, site_coords):
    y = df['latitude'] - site_coords[0]
    x = df['longitude'] - site_coords[1]
    theta = np.arctan2(y, x)/math.pi*180 + 180
    return theta

train['theta'] = train.apply(calc_degrees, site_coords=(center_lat, center_lon), axis=1)
test['theta'] = test.apply(calc_degrees, site_coords=(center_lat, center_lon), axis=1)

## Rublevka engineering

In [21]:
rublevka_lat = 55.7380095
rublevka_lon = 37.2588927
train['rublevka'] = train.apply(calc_distance, site_coords=(rublevka_lat, rublevka_lon), axis=1)
test['rublevka'] = test.apply(calc_distance, site_coords=(rublevka_lat, rublevka_lon), axis=1)

## Khamovniki engineering

In [22]:
Khamovniki_lat = 55.7297634
Khamovniki_lon = 37.5710506
train['Khamovniki'] = train.apply(calc_distance, site_coords=(Khamovniki_lat, Khamovniki_lon), axis=1)
test['Khamovniki'] = test.apply(calc_distance, site_coords=(Khamovniki_lat, Khamovniki_lon), axis=1)

## Patriarch Ponds engineering

In [23]:
patriach_ponds_lat = 55.7641852
patriach_ponds_lon = 37.5898885
train['patriarch_ponds'] = train.apply(calc_distance, site_coords=(patriach_ponds_lat, patriach_ponds_lon), axis=1)
test['patriarch_ponds'] = test.apply(calc_distance, site_coords=(patriach_ponds_lat, patriach_ponds_lon), axis=1)

## Metro engineering

In [24]:
df_metro = pd.read_csv('metro.csv', delimiter=',')
df_metro = df_metro.drop(['Snippet', 'description', 'altitude', 'name'], axis=1)

def generate_metro_distance(df_metro, df_data):
    coordinates = df_data[['longitude','latitude']]
    ary = scipy.spatial.distance.cdist(coordinates, df_metro, 'euclidean' )
    min_dist = pd.DataFrame(ary)
    min_dist = min_dist.min(axis=1)*1000
    min_dist = min_dist.reindex_like(df_data['id'], method ='ffill')
    df_data['metro_distance'] = min_dist*1000
    
generate_metro_distance(df_metro, test);
generate_metro_distance(df_metro, train);

## Park engineering

In [25]:
df_parks = pd.read_csv('parks.csv', delimiter=',')
df_parks = df_parks.drop(['name'], axis=1)

def generate_park_distance(df_metro, df_data):
    coordinates = df_data[['latitude','longitude']]
    ary = scipy.spatial.distance.cdist(coordinates, df_metro, 'euclidean' )
    min_dist = pd.DataFrame(ary)
    min_dist = min_dist.min(axis=1)*1000
    min_dist = min_dist.reindex_like(df_data['id'], method ='ffill')
    df_data['park_distance'] = min_dist*1000
    
generate_park_distance(df_parks, test);
generate_park_distance(df_parks, train);


## School engineering

In [26]:
df_schools = pd.read_csv('schools.csv', delimiter=',')
df_schools = df_schools.drop(['name'], axis=1).astype(float, errors = 'raise')

def generate_school_distance(df_schools, df_data):
    coordinates = df_data[['latitude','longitude']]
    ary = scipy.spatial.distance.cdist(coordinates, df_schools, 'euclidean' ) 
    min_dist = min_dist.min(axis=1)*1000
    min_dist = min_dist.reindex_like(df_data['id'], method ='ffill')
    df_data['school_distance'] = min_dist*1000
    
generate_school_distance(df_schools, test);
generate_school_distance(df_schools, train);

## Embassies engineering

In [27]:
df_embassies = pd.read_csv('embassies.csv', delimiter=',')
df_embassies= df_embassies.drop(['country'], axis=1).astype(float, errors = 'raise')

def generate_embassies_distance(df_embassies, df_data):
    coordinates = df_data[['latitude','longitude']]
    ary = scipy.spatial.distance.cdist(coordinates, df_embassies, 'euclidean' ) 
    min_dist = pd.DataFrame(ary)
    min_dist = min_dist.min(axis=1)*1000
    min_dist = min_dist.reindex_like(df_data['id'], method ='ffill')
    df_data['embassy_distance'] = min_dist*1000
    
generate_embassies_distance(df_embassies, test);
generate_embassies_distance(df_embassies, train);

## Age engineering

In [28]:
train['age'] = 2021 - train['constructed'].astype(float) 
test['age'] = 2021 - test['constructed'].astype(float)

## Relative Floor engineering

In [29]:
train['rel_floor'] = train['floor'] / train['stories'].astype(float)
test['rel_floor'] = test['floor'] / test['stories'].astype(float)

## Elevator and floor engineering

In [30]:
train['elevator_floor'] = train.apply(lambda df: df['floor']* -1 if df['elevator_without']== 1 else df['floor']* 1, axis=1)
test['elevator_floor'] = test.apply(lambda df: df['floor']* -1 if df['elevator_without']== 1 else df['floor']* 1, axis=1)

## Drop id

In [31]:
train = train.drop(['id'], axis=1);
test = test.drop(['id'], axis=1);

## Drop Building id

In [32]:
train = train.drop(['building_id'], axis=1);
test = test.drop(['building_id'], axis=1);

## Outliers

In [33]:
N_folds = 1
folds = [None] * N_folds

for fold in range(N_folds):
    x_train, x_test, Y_train, Y_test = train_test_split(train, train['price'], test_size=0.33)
    folds[fold] = [x_train, x_test, Y_train, Y_test]

model1 = xgb.XGBRegressor(
    colsample_bytree=0.2,              
    learning_rate=0.06,
    max_depth=8,
    n_estimators=1150,
    eval_metric='rmse'
);

model2 = RandomForestRegressor(
    n_estimators = 719,
    max_depth = 31,
    n_jobs = -1,
    random_state = 42
)

worst_rows = []
for fold in range(N_folds):
    X_train, X_test, y_train, y_test = folds[fold]
    
    model2.fit(X_train, y_train);
    lgbm_prediction = model2.predict(X_test)

    prediction = pd.Series(np.array(lgbm_prediction))
    prediction.index = y_test.index
    df_results = pd.concat([y_test.rename("Test"), prediction.rename("Prediction")], axis=1)
    df_results['Error'] = (df_results['Test'] - df_results['Prediction']).abs()
    df_results = df_results.sort_values(by=['Error'], ascending=False)
    
    worst_rows += df_results.head(100).index.tolist()
    
worst_rows = list(set(worst_rows))
train = train.drop(train.index[worst_rows])

## Saving modified data


In [34]:
# THIS SHOULD ALWAYS BE THE LAST CELL!
omit_cols = []
drop = ['price']
X_train = train.drop(drop + omit_cols, axis=1)
y_train = train['price'];
X_test = test.drop(omit_cols, axis=1)

%store X_train
%store y_train
%store X_test

%store df_train
%store df_test

Stored 'X_train' (DataFrame)
Stored 'y_train' (Series)
Stored 'X_test' (DataFrame)
Stored 'df_train' (DataFrame)
Stored 'df_test' (DataFrame)


# Model

In [35]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, KFold
import sklearn.model_selection as model_selection
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from scipy.special import inv_boxcox 


import lightgbm as lgb
from catboost import CatBoostRegressor
import xgboost as xgb

import matplotlib.pylab as plt
plt.style.use('ggplot')

In [36]:
%store -r X_train 
%store -r y_train 
%store -r X_test

%store -r df_train 
%store -r df_test

X_test_c = X_test.copy()

## True Stacking

In [37]:
ntrain = X_train.shape[0]
ntest = X_test.shape[0]
SEED = 42 # for reproducibility
NFOLDS = 5 # set number of folds for out-of-fold prediction
kf = KFold(
    n_splits=NFOLDS,
    shuffle=True,
    random_state=SEED
) # K-Folds cross-validator

def get_oof(clf, x_train, y_train, x_test):
    """
    Popular function on Kaggle.
    
    Trains a classifier on 4/5 of the training data and
    predicts the rest (1/5). This procedure is repeated for all 5 folds,
    thus we have predictions for all training set. This prediction is one
    column of meta-data, later on used as a feature column by a meta-algorithm.
    We predict the test part and average predictions across all 5 models.
    
    Keyword arguments:
    clf -- classifier
    x_train -- 4/5 of training data
    y_train -- corresponding labels
    x_test -- all test data
    
    """
    %store -r X_train
    
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    importance_array = np.zeros(len(X_train.columns.tolist()))
    
    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.fit(x_tr, y_tr)
        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)
        
        importance_array += clf.feature_importances_
        print(f"{round((i+1)/NFOLDS*100)}% Done")
    
    importance_array = importance_array / NFOLDS
    importance_array = importance_array / importance_array.sum()
    feature_importance = pd.DataFrame([X_train.columns, importance_array]).T.sort_values([1], ascending = [False])

    oof_test[:] = oof_test_skf.mean(axis=0)
    print("\n")

    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1), importance_array

In [38]:
model1 = DecisionTreeRegressor(
    criterion = 'squared_error',
    max_depth = 5,
    min_samples_split = 4,
    min_samples_leaf = 2,
    random_state=SEED
)

model2 = RandomForestRegressor(
    n_estimators = 500, # 719 for submission with score 0.15095
    max_depth = 23, # 31 for submission with score 0.15095
    n_jobs = -1,
    random_state = SEED
)

model3 = lgb.LGBMRegressor(
    num_leaves=10, 
    max_depth=5,
    verbose=-1,
    metric='squared_error',
    n_jobs=4, 
    n_estimators=1500, # 2000 for submission with score 0.15095
    colsample_bytree=0.95,
    subsample=0.9,
    learning_rate=0.005, 
    random_state = SEED
);

model4 = CatBoostRegressor(
    n_estimators=1250, # 1000 for submission with score 0.15095
    learning_rate=0.2,# 0.1 for submission with score 0.15095
    thread_count=-1,
    depth=7, 
    silent=True,
    bagging_temperature=0.2,
    random_seed=SEED,
)

model5 = xgb.XGBRegressor(
    eta =  0.03842638804730825,
    max_depth = 12,
    min_child_weight = 9.895364542533038,
    n_estimators = 965,
    n_jobs = -1,
    random_state = SEED
)

model6 = GradientBoostingRegressor(
    learning_rate=0.01,
    n_estimators=2000,
    subsample=1.0, 
    criterion='mse', 
    min_samples_split=4, 
    min_samples_leaf=2, 
    min_weight_fraction_leaf=0.0, 
    max_depth=9, 
    min_impurity_decrease=0.0, 
    init=None, 
    random_state=42, 
    max_features=None,
    alpha=0.9,
    verbose=0,
    max_leaf_nodes=None,
    warm_start=False,
    validation_fraction=0.1,
    n_iter_no_change=None,
    tol=0.0001,
    ccp_alpha=0.0
)

In [39]:
X_train = X_train.values
X_test = X_test.values
y_train = y_train.ravel()

In [40]:
print('OOF Model 1 starting')
dt_oof_train, dt_oof_test, importance_array_1 = get_oof(model1, X_train, y_train, X_test);
X_train = X_train.values

print('OOF Model 2 starting')
rf_oof_train, rf_oof_test, importance_array_2 = get_oof(model2, X_train, y_train, X_test)
X_train = X_train.values

print('OOF Model 3 starting')
lgbm_oof_train, lgbm_oof_test, importance_array_3 = get_oof(model3, X_train, y_train, X_test)
X_train = X_train.values

print('OOF Model 4 starting')
cat_oof_train, cat_oof_test, importance_array_4 = get_oof(model4, X_train, y_train, X_test)
X_train = X_train.values

print('OOF Model 5 starting')
xgb_oof_train, xgb_oof_test, importance_array_5 = get_oof(model5, X_train, y_train, X_test)
X_train = X_train.values

print('OOF Model 6 starting')
gb_oof_train, gb_oof_test, importance_array_6 = get_oof(model6, X_train, y_train, X_test)
X_train = X_train.values

OOF Model 1 starting
20% Done
40% Done
60% Done
80% Done
100% Done


OOF Model 2 starting
20% Done
40% Done
60% Done
80% Done
100% Done


OOF Model 3 starting
20% Done
40% Done
60% Done
80% Done
100% Done


OOF Model 4 starting
20% Done
40% Done
60% Done
80% Done
100% Done


OOF Model 5 starting
20% Done
40% Done
60% Done
80% Done
100% Done


OOF Model 6 starting
20% Done
40% Done
60% Done
80% Done
100% Done




In [41]:
x_train = np.concatenate((
    dt_oof_train,
    rf_oof_train,
    lgbm_oof_train,
    cat_oof_train,
    xgb_oof_train,
    gb_oof_train
), axis=1)

x_test = np.concatenate((
    dt_oof_test,
    rf_oof_test,
    lgbm_oof_test,
    cat_oof_test,
    xgb_oof_test,
    gb_oof_test
), axis=1)

In [42]:
META_MODEL = lgb.LGBMRegressor(
    num_leaves=5, 
    max_depth=8,
    random_state=SEED, 
    verbose=-1, 
    metric='mse',
    n_jobs=4, 
    n_estimators=300,
    colsample_bytree=1,
    subsample=0.9,
    learning_rate=0.08 # 0.05 for submission with score 0.15095
)
META_MODEL.fit(x_train, y_train)
final_predictions = META_MODEL.predict(x_test)

print("--- %s seconds ---" % (time.time() - start_time));

--- 3053.329996109009 seconds ---


In [43]:
%store -r X_train 
    
df_results = pd.DataFrame({ \
    'id': df_test['id'], \
    'price_prediction': final_predictions})

#Revert to exponential price
df_results['price_prediction'] = df_results['price_prediction'].apply(lambda x: 10**x);
df_results['price_prediction'] = df_results['price_prediction']*X_test_c['area_total'];
print(df_results)
df_results.to_csv( 'submission.csv', index=False)
    
models = ['DecisionTreeRegressor', 'RandomForestRegressor', 'LGBMRegressor', 'CatBoostRegressor', 'XGBRegressor', 'GradientBoostingRegressor']
feature_importance = pd.DataFrame([models, META_MODEL.feature_importances_]).T.sort_values([1], ascending = [False])
normalized_feature_importance = feature_importance[1] / feature_importance[1].sum()
display(feature_importance)
    
meta_importance = np.zeros(importance_array_1.size)
for i, importance in  enumerate(normalized_feature_importance):
    variable = vars()['importance_array_' + str(i+1)]
    meta_importance += importance * variable
feature_importance = pd.DataFrame([X_train.columns, meta_importance]).T.sort_values([1], ascending = [False])
display(feature_importance)  
print("Sum of stacked features:", feature_importance[1].sum())    

         id  price_prediction
0     23285      2.867222e+07
1     23286      9.402465e+06
2     23287      6.059168e+06
3     23288      8.747068e+06
4     23289      5.216476e+06
...     ...               ...
9932  33217      3.207355e+07
9933  33218      1.764460e+07
9934  33219      9.327347e+06
9935  33220      9.439049e+06
9936  33221      7.247058e+06

[9937 rows x 2 columns]


,0,1
2,LGBMRegressor,241
1,RandomForestRegressor,228
4,XGBRegressor,202
0,DecisionTreeRegressor,193
3,CatBoostRegressor,181
5,GradientBoostingRegressor,155


,0,1
34,Khamovniki,0.346244
1,area_total,0.0896801
31,distance,0.0440544
35,patriarch_ponds,0.0392091
20,street,0.0368286
21,constructed,0.0319633
38,school_distance,0.0262464
39,embassy_distance,0.0262182
36,metro_distance,0.0233354
23,stories,0.0230347


Sum of stacked features: 1.0000000000000002
